# Препроцессинг

In [3]:
import openpyxl
import re
import pymystem3
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import gensim
import logging
import nltk.data
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import word2vec
import warnings
warnings.filterwarnings('ignore')

# Парсер корпуса

In [4]:
import pickle
from pprint import pprint

In [5]:
with open('/Volumes/Анна/Corpora/stable/disamb/corpora/lines_1m_to_3m.pickle', 'rb') as f:
    data_13 = pickle.load(f)

In [6]:
with open('/Volumes/Анна/Corpora/stable/disamb/corpora/lines_3000000.pickle', 'rb') as f:
    data_3 = pickle.load(f)

In [7]:
BigData = data_13 + data_3

In [8]:
# Видим точку не в т.к., т.д. или т.е. - новое предложение
text = []  # Список предложений
sentence = []  # Обновляемый список слов в предложении
k = -1  # Счетчик позиции. Вот если у нас "т" из т.е., мы можем не обрабатывать следующие 3 символа
for i in range(len(BigData)-2):
    if i <= k:  # Если счетчик больше, то мы уже учли это слово
        continue
    elif len(BigData[i]) != 7:
        continue
    else:
        k = -1
        if not (BigData[i][0] == '!' or BigData[i][0] == '?' or BigData[i][0] == '.' or BigData[i][0] == '…') :
            if ((BigData[i][0] == 'то' or BigData[i][0] == 'То') 
                and BigData[i][6] == 'предик' and BigData[i+1][0] == 'есть'):
                sentence.append('тоесть')
                k = i + 1  # Не смотрим на следующее "есть"
            elif (BigData[i][0] == 'так' or BigData[i][0] == 'Так') and BigData[i+1][0] == 'как':
                sentence.append('таккак')
                k = i + 1
            elif (BigData[i][0] == 'потому' or BigData[i][0] == 'Потому') and BigData[i+1][0] == 'что':
                sentence.append('потомучто')
                k = i + 1
            elif (BigData[i][0] == 'в' or BigData[i][0] == 'В') and BigData[i+1][0] == 'смысле':
                sentence.append('всмысле')  # их всего 156
                k = i + 1
            elif (BigData[i][0] == 'А' or BigData[i][0] == 'А') and BigData[i+1][0] == 'именно':
                sentence.append('аименно')
                k = i + 1
            elif ((BigData[i][0] == 'т' or BigData[i][0] == 'Т') 
                  and BigData[i+1][0] == '.' and BigData[i+2][0] == 'е'):
                sentence.append('тоесть')
                k = i + 3 
            elif (BigData[i][0] == 'т' or BigData[i][0] == 'Т') and BigData[i+1][0] == '.' and BigData[i+2][0] == 'к':
                sentence.append('таккак')
                k = i + 3
            elif BigData[i][6] == 'PUNC' or BigData[i][0] == '–':  # Если не !.?... но знак препинания, не добавляем
                continue
            else:
                sentence.append(BigData[i][2])
        elif not (BigData[i+1][0] == 'п' or BigData[i+1][0] == 'д'):  # Если у нас точка, и она не входит в "т.п." и "т.д."
            text.append(sentence)
            sentence = []

In [9]:
with open('lems.txt', 'w') as fw:
    for i in text:
        fw.write(' '.join(i).lower())
        fw.write('/n')

# Word2Vec CBOW

In [10]:
f = 'lems.txt'
data = gensim.models.word2vec.LineSentence(f)

In [11]:
model_cbow = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50)

In [12]:
model_cbow.init_sims(replace=True)
model_path = "to.bin"

print("Saving model...")
model_cbow.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [13]:
pprint(model_cbow.wv.most_similar("тоесть", topn=50))

[('же', 0.2339174598455429),
 ('человек', 0.22926689684391022),
 ('установить', 0.21754902601242065),
 ('уж', 0.21670973300933838),
 ('который', 0.21562625467777252),
 ('порядок', 0.21257881820201874),
 ('но', 0.20823979377746582),
 ('политический', 0.20551401376724243),
 ('вот', 0.2051548957824707),
 ('а', 0.2025071382522583),
 ('должно', 0.2024683654308319),
 ('и', 0.20115657150745392),
 ('быть', 0.20059838891029358),
 ('то', 0.20045281946659088),
 ('далее', 0.19793909788131714),
 ('что', 0.19752058386802673),
 ('вещество', 0.19208024442195892),
 ('в', 0.1920768916606903),
 ('воздействие', 0.19192764163017273),
 ('значить', 0.18879765272140503),
 ('всегда', 0.1868061125278473),
 ('действие', 0.18424606323242188),
 ('при', 0.18316248059272766),
 ('дело', 0.1831446886062622),
 ('если', 0.1812722533941269),
 ('счет', 0.18114957213401794),
 ('путь', 0.1800999939441681),
 ('таккак', 0.17794837057590485),
 ('скорее', 0.17755836248397827),
 ('глубокий', 0.17682623863220215),
 ('сделать', 0.

In [112]:
l = 'я здесь самый умный тоесть как  самый умный  недоумевать'

In [113]:
l = l.split()

In [114]:
model_cbow.predict_output_word(l) # Это я не очень поняла как работает, но еще особенно не разбиралась

[('лучший', 3.6221896e-05),
 ('главное', 3.6005662e-05),
 ('дело', 3.525131e-05),
 ('важный', 3.524584e-05),
 ('умный', 3.4789413e-05),
 ('красивый', 3.4572855e-05),
 ('интересный', 3.426596e-05),
 ('слово', 3.350955e-05),
 ('смелый', 3.3074713e-05),
 ('правильный', 3.3052063e-05)]

## Дальше я пробовала работать с другими моделями

Тут все плоховато

# CBOW с моими примерами

In [15]:
with open("21_century-Copy1.txt", "a") as fw:
    for i in text:
        fw.write(' '.join(i).lower())
        fw.write('/n')

In [16]:
f = '21_century-Copy1.txt'
data = gensim.models.word2vec.LineSentence(f)

In [17]:
model_cbow_full = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50)

In [18]:
model_cbow_full.init_sims(replace=True)
model_path = "to.bin"

print("Saving model...")
model_cbow_full.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [189]:
from gensim.models.keyedvectors import KeyedVectors
MODEL = KeyedVectors.load_word2vec_format('to.bin', binary=True)

In [35]:
pprint(model_cbow_full.wv.most_similar("тоесть", topn=50))

[('который', 0.2940874695777893),
 ('в', 0.2801324725151062),
 ('и', 0.27532321214675903),
 ('но', 0.2611147463321686),
 ('более', 0.250515341758728),
 ('система', 0.24847330152988434),
 ('весь', 0.24701091647148132),
 ('быть', 0.2457403838634491),
 ('вещество', 0.24558228254318237),
 ('или', 0.2443869411945343),
 ('воздействие', 0.24421289563179016),
 ('то', 0.24326325953006744),
 ('иметь', 0.24057117104530334),
 ('обусловить', 0.2402734011411667),
 ('человек', 0.23677486181259155),
 ('развитие', 0.23260119557380676),
 ('их', 0.23105686902999878),
 ('человеческий', 0.22861215472221375),
 ('для', 0.22795844078063965),
 ('же', 0.22307217121124268),
 ('а', 0.22192877531051636),
 ('необходимость', 0.22090625762939453),
 ('получать', 0.22060106694698334),
 ('значит', 0.21984604001045227),
 ('поддержание', 0.2176116555929184),
 ('«', 0.21673578023910522),
 ('поэтому', 0.2156844139099121),
 ('как', 0.21504560112953186),
 ('реформа', 0.2148822844028473),
 ('на', 0.21421800553798676),
 ('при',

# Word2Vec Skip-gram

Ничего не работает, потому что это skip-gram

In [20]:
f = '21_century-Copy1.txt'
data = gensim.models.word2vec.LineSentence(f)

In [21]:
model_sg = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50, sg=1)

In [22]:
model_sg.init_sims(replace=True)
model_path = "to.bin"

print("Saving model...")
model_sg.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [23]:
model_sg.wv.most_similar("тоесть", topn=50)

[('невозможно/nтоесть', 0.33740538358688354),
 ('прогрессивнее', 0.3313928246498108),
 ('11-13', 0.3291909694671631),
 ('уровень/nи', 0.3153378367424011),
 ('и', 0.3144078254699707),
 ('спг', 0.31358468532562256),
 ('уклад', 0.31152862310409546),
 ('новoгo', 0.31128817796707153),
 ('прцент/n)', 0.30413293838500977),
 ('ароматно', 0.3036803901195526),
 ('общенародный', 0.30315157771110535),
 ('а', 0.3015827238559723),
 ('обтекаемый', 0.2996218800544739),
 ('тыс', 0.2992943227291107),
 ('воздействие', 0.29762357473373413),
 ('отчисление', 0.2953924536705017),
 ('бездействовать', 0.2939915060997009),
 ('но', 0.29247188568115234),
 ('сколько-нибудь', 0.2914343476295471),
 ('bs', 0.2910422682762146),
 ('вливать', 0.28934359550476074),
 ('усвоение', 0.28910285234451294),
 ('изогнутый', 0.2890014946460724),
 ('читаемость', 0.2876467704772949),
 ('млн', 0.2873305678367615),
 ('способность/nв', 0.28715139627456665),
 ('тор-м1', 0.28462672233581543),
 ('высвобождаться', 0.2843358814716339),
 ('и

# GloVe

Тут все совсем странно

In [24]:
with open('21_century-Copy1.txt') as fh:
    lines = fh.readlines()
lines = [line.split() for line in lines]

In [25]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lines, window=10)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=5, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [26]:
print(glove.most_similar('тоесть'))

[('дело/nи', 0.9995947446055201), ('появиться', 0.9989923279255014), ('например', 0.9989316235175412), ('весь', 0.9987825329634878)]


# AdaGram

Во втором значении модель говорит, что *то есть* похоже на какие-то редкие полнозначные слова. В первом значении немного получше.

In [27]:
! adagram-train 21_century-Copy1.txt disamb.pkl

[INFO] 2020-02-03 09:40:19,500 Building dictionary...
[INFO] 2020-02-03 09:45:00,428 Done! 21138 words.
[INFO] 2020-02-03 09:49:34,040 1.98% -7.3071 0.0245 1.2/2.0 0.23 kwords/sec
[INFO] 2020-02-03 09:49:37,808 3.96% -7.1314 0.0240 1.3/3.0 16.98 kwords/sec
[INFO] 2020-02-03 09:49:41,586 5.94% -6.9853 0.0235 1.5/4.0 16.94 kwords/sec
[INFO] 2020-02-03 09:49:45,349 7.92% -6.8650 0.0230 1.6/5.0 17.00 kwords/sec
[INFO] 2020-02-03 09:49:49,035 9.90% -6.7619 0.0225 1.6/5.0 17.36 kwords/sec
[INFO] 2020-02-03 09:49:52,631 11.88% -6.6703 0.0220 1.7/5.0 17.80 kwords/sec
[INFO] 2020-02-03 09:49:56,166 13.86% -6.5874 0.0215 1.7/5.0 18.11 kwords/sec
[INFO] 2020-02-03 09:49:59,669 15.84% -6.5116 0.0210 1.7/5.0 18.27 kwords/sec
[INFO] 2020-02-03 09:50:03,148 17.83% -6.4419 0.0205 1.8/5.0 18.39 kwords/sec
[INFO] 2020-02-03 09:50:06,610 19.81% -6.3778 0.0200 1.8/5.0 18.49 kwords/sec
[INFO] 2020-02-03 09:50:10,069 21.79% -6.3186 0.0196 1.8/5.0 18.50 kwords/sec
[INFO] 2020-02-03 09:50:13,496 23.77% -6.264

In [28]:
import adagram
vm = adagram.VectorModel.load('disamb.pkl')

In [29]:
vm.word_sense_probs('тоесть')

[(0, 0.9999454178265379)]

In [34]:
vm.sense_neighbors('тоесть', 1)

[('кнут', 0, 0.4049072),
 ('переулок', 0, 0.35147887),
 ('старое', 0, 0.341325),
 ('дегустация', 0, 0.3382282),
 ('завязываться', 0, 0.3379058),
 ('конце-концов', 0, 0.33070037),
 ('темнеть', 0, 0.32989046),
 ('более-менее', 0, 0.32591155),
 ('видеть', 0, 0.32531103),
 ('положить', 0, 0.32296586)]

In [193]:
from vec2graph import visualize
visualize('./', MODEL, 'тоесть', depth=0, topn=10, threshold=0.3, edge=1, sep=True)

Visualizations written to ./


['тоесть']

In [ ]:
depth=0, topn=8, threshold=0.62, edge=1, sep=True)